In [1]:
# relative import ../build/fast_kinematics
import sys
sys.path.append('../build')
import fast_kinematics
import numpy as np
import time
import torch
import pytorch_kinematics as pk

In [101]:
N = 4096

In [102]:
model = fast_kinematics.FastKinematics("../kuka_iiwa.urdf",N,"lbr_iiwa_link_7")
qpos = np.random.rand(7*N)

all_times = []
for _ in range(1000):
  start_time = time.time()
  poses = np.array(model.jacobian_mixed_frame(qpos))
  poses = poses.reshape(N,-1,6).transpose(0,2,1)
  end_time = time.time()
  all_times.append(end_time - start_time)
print(f"Time taken for jacobian_mixed_frame: {np.mean(all_times)} with std: {np.std(all_times)}")

Time taken for jacobian_mixed_frame: 0.009627951383590698 with std: 0.0021229041694825683


In [103]:
chain = pk.build_serial_chain_from_urdf(open("../kuka_iiwa.urdf").read(), "lbr_iiwa_link_7")
d = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.float32

chain = chain.to(dtype=dtype, device=d)
th = torch.rand(N, 7, dtype=dtype, device=d, requires_grad=True)
all_times = []
for _ in range(100):
  start_time = time.time()
  J = chain.jacobian(th)
  end_time = time.time()
  all_times.append(end_time - start_time)
print(f"Time taken for jacobian_mixed_frame: {np.mean(all_times)} with std: {np.std(all_times)}")

Time taken for jacobian_mixed_frame: 0.03035259246826172 with std: 0.003549135053997798


In [104]:
model = fast_kinematics.FastKinematics("../kuka_iiwa.urdf",N,"lbr_iiwa_link_7")
qpos = np.random.rand(7*N)

all_times = []
for _ in range(100):
  start_time = time.time()
  poses = model.do_nothing(qpos)
  # poses = poses.reshape(N,-1,6).transpose(0,2,1)
  end_time = time.time()
  all_times.append(end_time - start_time)
print(f"Time taken for jacobian_mixed_frame: {np.mean(all_times)} with std: {np.std(all_times)}")

Time taken for jacobian_mixed_frame: 0.008483309745788575 with std: 0.001416382877823726


In [105]:
qpos = qpos.reshape(N*7,1)
poses = model.forward_kinematics(qpos)
print(poses.shape)

(28672,)
